In [3]:
# -*- coding: utf-8 -*-
"""
Created on 05/13, 2023
@author: WillyF

"""
# https://data.world/etocco/nba-player-stats



'\nCreated on 05/13, 2023\n@author: WillyF\n\n'

In [4]:
import pandas as pd
import urllib.request
import json
import numpy as np
# from glob import glob
import os
from warnings import filterwarnings
filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [20]:
nba_team_list = pd.read_csv('NBA_Team_List.csv').fillna(0.0)
nba_player_df = pd.read_csv('https://query.data.world/s/ty4piswbrsp54noyh2bdvrjecv24ta?dws=00000').fillna(0.0)
nba_player_df["BHOF"] = False
nba_player_df.loc[nba_player_df["Player"].str.contains("*", regex=False),"BHOF"] = True
nba_player_df["Player"] = nba_player_df["Player"].str.strip("*")
nba_player_df = nba_player_df.merge(nba_team_list, how="left", on="Tm", validate="many_to_one")
nba_player_df

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season,MVP,BHOF,Team
0,1,Mahmoud Abdul-Rauf,PG,28,SAC,31,0,17.1,3.3,8.8,0.377,0.2,1.0,0.161,3.2,7.8,0.405,0.386,0.5,0.5,1.000,0.2,1.0,1.2,1.9,0.5,0.0,0.6,1.0,7.3,1997-98,False,False,Sacramento Kings
1,2,Tariq Abdul-Wahad,SG,23,SAC,59,16,16.3,2.4,6.1,0.403,0.1,0.3,0.211,2.4,5.7,0.414,0.409,1.4,2.1,0.672,0.7,1.2,2.0,0.9,0.6,0.2,1.1,1.4,6.4,1997-98,False,False,Sacramento Kings
2,3,Shareef Abdur-Rahim,SF,21,VAN,82,82,36.0,8.0,16.4,0.485,0.3,0.6,0.412,7.7,15.8,0.488,0.493,6.1,7.8,0.784,2.8,4.3,7.1,2.6,1.1,0.9,3.1,2.5,22.3,1997-98,False,False,Memphis Grizzlies
3,4,Cory Alexander,PG,24,TOT,60,22,21.6,2.9,6.7,0.428,1.1,2.9,0.375,1.8,3.7,0.469,0.510,1.3,1.7,0.784,0.3,2.2,2.4,3.5,1.2,0.2,1.9,1.6,8.1,1997-98,False,False,Two Other Teams
4,4,Cory Alexander,PG,24,SAS,37,3,13.5,1.6,3.9,0.414,0.5,1.7,0.313,1.1,2.2,0.494,0.483,0.7,1.0,0.676,0.2,1.1,1.3,1.9,0.7,0.1,1.3,1.4,4.5,1997-98,False,False,San Antonio Spurs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14568,601,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,0.465,0.7,1.7,0.395,1.9,3.9,0.495,0.524,0.5,1.0,0.481,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2021-22,False,False,Toronto Raptors
14569,602,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,0.460,3.1,8.0,0.382,6.3,12.3,0.512,0.536,6.6,7.3,0.904,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2021-22,False,False,Atlanta Hawks
14570,603,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,0.526,0.0,0.2,0.091,2.3,4.2,0.547,0.528,0.7,1.1,0.623,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2021-22,False,False,Miami Heat
14571,604,Cody Zeller,C,29,POR,27,0,13.1,1.9,3.3,0.567,0.0,0.1,0.000,1.9,3.2,0.593,0.567,1.4,1.8,0.776,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2021-22,False,False,Portland Trail Blazers


In [21]:
nba_player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14573 entries, 0 to 14572
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      14573 non-null  int64  
 1   Player  14573 non-null  object 
 2   Pos     14573 non-null  object 
 3   Age     14573 non-null  int64  
 4   Tm      14573 non-null  object 
 5   G       14573 non-null  int64  
 6   GS      14573 non-null  int64  
 7   MP      14573 non-null  float64
 8   FG      14573 non-null  float64
 9   FGA     14573 non-null  float64
 10  FG%     14573 non-null  float64
 11  3P      14573 non-null  float64
 12  3PA     14573 non-null  float64
 13  3P%     14573 non-null  float64
 14  2P      14573 non-null  float64
 15  2PA     14573 non-null  float64
 16  2P%     14573 non-null  float64
 17  eFG%    14573 non-null  float64
 18  FT      14573 non-null  float64
 19  FTA     14573 non-null  float64
 20  FT%     14573 non-null  float64
 21  ORB     14573 non-null  float64
 22

In [23]:
nba_player_df[nba_player_df["Player"]=="LeBron James"]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season,MVP,BHOF,Team
3263,203,LeBron James,SG,19,CLE,79,79,39.5,7.9,18.9,0.417,0.8,2.7,0.290,7.1,16.1,0.438,0.438,4.4,5.8,0.754,1.3,4.2,5.5,5.9,1.6,0.7,3.5,1.9,20.9,2003-04,False,False,Cleveland Cavaliers
3851,215,LeBron James,SF,20,CLE,80,80,42.4,9.9,21.1,0.472,1.4,3.9,0.351,8.6,17.2,0.499,0.504,6.0,8.0,0.750,1.4,6.0,7.4,7.2,2.2,0.7,3.3,1.8,27.2,2004-05,False,False,Cleveland Cavaliers
4415,207,LeBron James,SF,21,CLE,79,79,42.5,11.1,23.1,0.480,1.6,4.8,0.335,9.5,18.3,0.518,0.515,7.6,10.3,0.738,0.9,6.1,7.0,6.6,1.6,0.8,3.3,2.3,31.4,2005-06,False,False,Cleveland Cavaliers
4947,207,LeBron James,SF,22,CLE,78,78,40.9,9.9,20.8,0.476,1.3,4.0,0.319,8.6,16.8,0.513,0.507,6.3,9.0,0.698,1.1,5.7,6.7,6.0,1.6,0.7,3.2,2.2,27.3,2006-07,False,False,Cleveland Cavaliers
5494,211,LeBron James,SF,23,CLE,75,74,40.4,10.6,21.9,0.484,1.5,4.8,0.315,9.1,17.1,0.531,0.518,7.3,10.3,0.712,1.8,6.1,7.9,7.2,1.8,1.1,3.4,2.2,30.0,2007-08,False,False,Cleveland Cavaliers
6078,208,LeBron James,SF,24,CLE,81,81,37.7,9.7,19.9,0.489,1.6,4.7,0.344,8.1,15.2,0.535,0.530,7.3,9.4,0.780,1.3,6.3,7.6,7.2,1.7,1.1,3.0,1.7,28.4,2008-09,True,False,Cleveland Cavaliers
6664,222,LeBron James,SF,25,CLE,76,76,39.0,10.1,20.1,0.503,1.7,5.1,0.333,8.4,15.0,0.560,0.545,7.8,10.2,0.767,0.9,6.4,7.3,8.6,1.6,1.0,3.4,1.6,29.7,2009-10,True,False,Cleveland Cavaliers
7259,226,LeBron James,SF,26,MIA,79,79,38.8,9.6,18.8,0.510,1.2,3.5,0.330,8.4,15.3,0.552,0.541,6.4,8.4,0.759,1.0,6.5,7.5,7.0,1.6,0.6,3.6,2.1,26.7,2010-11,False,False,Miami Heat
7833,234,LeBron James,SF,27,MIA,62,62,37.5,10.0,18.9,0.531,0.9,2.4,0.362,9.1,16.5,0.556,0.554,6.2,8.1,0.771,1.5,6.4,7.9,6.2,1.9,0.8,3.4,1.5,27.1,2011-12,True,False,Miami Heat
8361,213,LeBron James,PF,28,MIA,76,76,37.9,10.1,17.8,0.565,1.4,3.3,0.406,8.7,14.5,0.602,0.603,5.3,7.0,0.753,1.3,6.8,8.0,7.3,1.7,0.9,3.0,1.4,26.8,2012-13,True,False,Miami Heat


In [11]:
set(df["Tm"])

{'ATL',
 'BOS',
 'BRK',
 'CHA',
 'CHH',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NJN',
 'NOH',
 'NOK',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'SEA',
 'TOR',
 'TOT',
 'UTA',
 'VAN',
 'WAS'}

In [6]:
set(df["Season"])

{'1997-98',
 '1998-99',
 '1999-00',
 '2000-01',
 '2001-02',
 '2002-03',
 '2003-04',
 '2004-05',
 '2005-06',
 '2006-07',
 '2007-08',
 '2008-09',
 '2009-10',
 '2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20',
 '2020-21',
 '2021-22'}